In [1]:
'''import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from keras.models import load_model

class HelmetDetectionApp:
    def __init__(self, root, video_source):
        self.root = root
        self.root.title("Helmet Detection App")

        self.video_source = video_source
        self.cap = cv2.VideoCapture(self.video_source)

        self.width = int(self.cap.get(3))
        self.height = int(self.cap.get(4))

        self.create_widgets()

        self.update()

    def create_widgets(self):
        self.label = ttk.Label(self.root, text="Helmet Detection")
        self.label.pack(padx=10, pady=10)

        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height)
        self.canvas.pack()

        self.quit_button = ttk.Button(self.root, text="Quit", command=self.quit)
        self.quit_button.pack(pady=10)

        # Load YOLOv3
        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")

        # Load helmet-nonhelmet model
        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")

    def update(self):
        ret, frame = self.cap.read()

        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # YOLOv3 detection
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            self.net.setInput(blob)
            outs = self.net.forward(self.net.getUnconnectedOutLayersNames())

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5 and class_id == 0:  # Class ID for helmet (adjust if needed)
                        center_x = int(detection[0] * self.width)
                        center_y = int(detection[1] * self.height)
                        w = int(detection[2] * self.width)
                        h = int(detection[3] * self.height)

                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        # Extract helmet region from the frame
                        helmet_roi = frame[y:y + h, x:x + w]

                        # Preprocess helmet_roi for helmet-nonhelmet model
                        helmet_roi_resized = cv2.resize(helmet_roi, (224, 224))  # Resize to match model input shape
                        helmet_roi_normalized = helmet_roi_resized / 255.0
                        helmet_roi_final = np.expand_dims(helmet_roi_normalized, axis=0)

                        # Predict helmet or nonhelmet
                        prediction = self.helmet_model.predict(helmet_roi_final)[0]

                        # Draw bounding box and label
                        label = "Helmet" if prediction[0] > 0.5 else "No Helmet"
                        color = (0, 255, 0) if label == "Helmet" else (0, 0, 255)
                        cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame_rgb, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            self.photo = ImageTk.PhotoImage(image=Image.fromarray(frame_rgb))
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

        self.root.after(10, self.update)

    def quit(self):
        self.cap.release()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = HelmetDetectionApp(root, video_source="stock-footage-india-goa-local-indian-people-ride-motor-bike-lot-native-hindu-drive-motorcycle-india.webm")
    root.mainloop()
'''

'import cv2\nimport numpy as np\nimport tkinter as tk\nfrom tkinter import ttk\nfrom PIL import Image, ImageTk\nfrom keras.models import load_model\n\nclass HelmetDetectionApp:\n    def __init__(self, root, video_source):\n        self.root = root\n        self.root.title("Helmet Detection App")\n\n        self.video_source = video_source\n        self.cap = cv2.VideoCapture(self.video_source)\n\n        self.width = int(self.cap.get(3))\n        self.height = int(self.cap.get(4))\n\n        self.create_widgets()\n\n        self.update()\n\n    def create_widgets(self):\n        self.label = ttk.Label(self.root, text="Helmet Detection")\n        self.label.pack(padx=10, pady=10)\n\n        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height)\n        self.canvas.pack()\n\n        self.quit_button = ttk.Button(self.root, text="Quit", command=self.quit)\n        self.quit_button.pack(pady=10)\n\n        # Load YOLOv3\n        self.net = cv2.dnn.readNet("yolov3-custom.

In [2]:
'''import cv2
import numpy as np
from keras.models import load_model
import tkinter as tk
from PIL import Image, ImageTk

class HelmetDetectionApp:
    def __init__(self, video_source):
        self.video_source = video_source
        self.cap = cv2.VideoCapture(self.video_source)

        self.frame_width = 500
        self.frame_height = 500

        # Load YOLOv3
        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")

        # Load helmet-nonhelmet model
        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")

        self.root = tk.Tk()
        self.root.title("Helmet Detection")

        self.label = tk.Label(self.root)
        self.label.pack()

        self.update()

        self.root.mainloop()

    def update(self):
        ret, frame = self.cap.read()

        if ret:
            # Resize frame for speedup
            frame = cv2.resize(frame, (self.frame_width, self.frame_height))
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # YOLOv3 detection
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            self.net.setInput(blob)
            outs = self.net.forward(self.net.getUnconnectedOutLayersNames())

            helmet_detected = False  # Flag to check if a helmet is detected

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5 and class_id == 0:  # Class ID for helmet (adjust if needed)
                        x, y, w, h = self.get_bbox_coordinates(detection)

                        # Check if width and height are positive
                        if w > 0 and h > 0:
                            # Extract helmet region from the frame
                            helmet_roi = frame[y:y + h, x:x + w]

                            # Check if the helmet_roi is not empty
                            if not helmet_roi.size == 0:
                                # Preprocess helmet_roi for helmet-nonhelmet model
                                helmet_roi_resized = cv2.resize(helmet_roi, (224, 224))
                                helmet_roi_normalized = helmet_roi_resized / 255.0
                                helmet_roi_final = np.expand_dims(helmet_roi_normalized, axis=0)

                                # Predict helmet or nonhelmet
                                prediction = self.helmet_model.predict(helmet_roi_final)[0]

                                # Draw bounding box and label
                                label = "Helmet" if prediction[0] > 0.5 else "No Helmet"
                                color = (0, 255, 0) if label == "Helmet" else (0, 0, 255)
                                cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 2)
                                cv2.putText(frame_rgb, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                                helmet_detected = True

            # Convert the frame to a format compatible with Tkinter
            image = Image.fromarray(frame_rgb)
            image = ImageTk.PhotoImage(image=image)

            # Update the label with the new image
            self.label.config(image=image)
            self.label.image = image

        # Schedule the next update after 10 milliseconds
        self.root.after(10, self.update)

    def get_bbox_coordinates(self, detection):
        center_x, center_y, w, h = int(detection[0] * self.frame_width), int(detection[1] * self.frame_height), \
                                   int(detection[2] * self.frame_width), int(detection[3] * self.frame_height)
        x, y = int(center_x - w / 2), int(center_y - h / 2)
        return x, y, w, h

if __name__ == "__main__":
    app = HelmetDetectionApp(video_source="stock-footage-india-goa-local-indian-people-ride-motor-bike-lot-native-hindu-drive-motorcycle-india (1).webm")
'''

'import cv2\nimport numpy as np\nfrom keras.models import load_model\nimport tkinter as tk\nfrom PIL import Image, ImageTk\n\nclass HelmetDetectionApp:\n    def __init__(self, video_source):\n        self.video_source = video_source\n        self.cap = cv2.VideoCapture(self.video_source)\n\n        self.frame_width = 500\n        self.frame_height = 500\n\n        # Load YOLOv3\n        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")\n\n        # Load helmet-nonhelmet model\n        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")\n\n        self.root = tk.Tk()\n        self.root.title("Helmet Detection")\n\n        self.label = tk.Label(self.root)\n        self.label.pack()\n\n        self.update()\n\n        self.root.mainloop()\n\n    def update(self):\n        ret, frame = self.cap.read()\n\n        if ret:\n            # Resize frame for speedup\n            frame = cv2.resize(frame, (self.frame_width, self.frame_height))\n            frame_r

In [3]:
'''import cv2
import numpy as np
from keras.models import load_model
import tkinter as tk
from PIL import Image, ImageTk

class HelmetDetectionApp:
    def __init__(self, video_source):
        self.video_source = video_source
        self.cap = cv2.VideoCapture(self.video_source)

        self.frame_width = 500
        self.frame_height = 500

        # Load YOLOv3
        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")

        # Load helmet-nonhelmet model
        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")

        self.root = tk.Tk()
        self.root.title("Helmet Detection")

        self.label = tk.Label(self.root)
        self.label.pack()

        # Folder to save snapshots
        self.output_folder = "C:\\Users\\Asus\\Videos\\Captures\\New folder"

        self.bike_detected = {}  # Dictionary to track detected bikes

        self.update()

        self.root.mainloop()

    def update(self):
        ret, frame = self.cap.read()

        if ret:
            # Resize frame for speedup
            frame = cv2.resize(frame, (self.frame_width, self.frame_height))
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # YOLOv3 detection
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            self.net.setInput(blob)
            outs = self.net.forward(self.net.getUnconnectedOutLayersNames())

            helmet_detected = False  # Flag to check if a helmet is detected

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5 and class_id == 0:  # Class ID for helmet (adjust if needed)
                        x, y, w, h = self.get_bbox_coordinates(detection)

                        # Check if width and height are positive
                        if w > 0 and h > 0:
                            # Extract helmet region from the frame
                            helmet_roi = frame[y:y + h, x:x + w]

                            # Check if the helmet_roi is not empty
                            if not helmet_roi.size == 0:
                                # Preprocess helmet_roi for helmet-nonhelmet model
                                helmet_roi_resized = cv2.resize(helmet_roi, (224, 224))
                                helmet_roi_normalized = helmet_roi_resized / 255.0
                                helmet_roi_final = np.expand_dims(helmet_roi_normalized, axis=0)

                                # Predict helmet or nonhelmet
                                prediction = self.helmet_model.predict(helmet_roi_final)[0]

                                # Draw bounding box and label
                                label = "Helmet" if prediction[0] > 0.5 else "No Helmet"
                                color = (0, 255, 0) if label == "Helmet" else (0, 0, 255)
                                cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 2)
                                cv2.putText(frame_rgb, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                                helmet_detected = True

                                # Store the image if a bike is detected without a helmet and it's not already stored
                                if label == "No Helmet" and self.bike_detected.get(label) is None:
                                    cv2.imwrite(
                                        f"{self.output_folder}/bike_no_helmet_{int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))}.png",
                                        frame)
                                    self.bike_detected[label] = True

            # Reset the dictionary if no helmet is detected
            if not helmet_detected:
                self.bike_detected = {}

            # Convert the frame to a format compatible with Tkinter
            image = Image.fromarray(frame_rgb)
            image = ImageTk.PhotoImage(image=image)

            # Update the label with the new image
            self.label.config(image=image)
            self.label.image = image

        # Schedule the next update after 10 milliseconds
        self.root.after(10, self.update)

    def get_bbox_coordinates(self, detection):
        center_x, center_y, w, h = int(detection[0] * self.frame_width), int(detection[1] * self.frame_height), \
                                   int(detection[2] * self.frame_width), int(detection[3] * self.frame_height)
        x, y = int(center_x - w / 2), int(center_y - h / 2)
        return x, y, w, h

if __name__ == "__main__":
    app = HelmetDetectionApp(video_source="video.mp4")
'''

'import cv2\nimport numpy as np\nfrom keras.models import load_model\nimport tkinter as tk\nfrom PIL import Image, ImageTk\n\nclass HelmetDetectionApp:\n    def __init__(self, video_source):\n        self.video_source = video_source\n        self.cap = cv2.VideoCapture(self.video_source)\n\n        self.frame_width = 500\n        self.frame_height = 500\n\n        # Load YOLOv3\n        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")\n\n        # Load helmet-nonhelmet model\n        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")\n\n        self.root = tk.Tk()\n        self.root.title("Helmet Detection")\n\n        self.label = tk.Label(self.root)\n        self.label.pack()\n\n        # Folder to save snapshots\n        self.output_folder = "C:\\Users\\Asus\\Videos\\Captures\\New folder"\n\n        self.bike_detected = {}  # Dictionary to track detected bikes\n\n        self.update()\n\n        self.root.mainloop()\n\n    def update(self):\n     

In [4]:
'''import cv2
import numpy as np
from keras.models import load_model
import tkinter as tk
from PIL import Image, ImageTk

class HelmetDetectionApp:
    """Application for detecting helmets in a video stream."""
    HELMET_CLASS_ID = 0  # Update with the correct class ID for helmets

    def __init__(self, video_source):
        self.video_source = video_source
        self.cap = cv2.VideoCapture(self.video_source)

        self.frame_width = 500
        self.frame_height = 500

        # Load YOLOv3
        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")

        # Load helmet-nonhelmet model
        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")

        self.root = tk.Tk()
        self.root.title("Helmet Detection")

        self.label = tk.Label(self.root)
        self.label.pack()

        # Folder to save snapshots
        self.output_folder = "C:\\Users\\Asus\\Videos\\Captures\\New folder"

        self.bike_detected = {}  # Dictionary to track detected bikes

        self.update()

        self.root.mainloop()

    def update(self):
        ret, frame = self.cap.read()

        if ret:
            frame_rgb = self.process_frame(frame)

            # Convert the frame to a format compatible with Tkinter
            image = Image.fromarray(frame_rgb)
            image = ImageTk.PhotoImage(image=image)

            # Update the label with the new image
            self.label.config(image=image)
            self.label.image = image

        # Schedule the next update after 10 milliseconds
        self.root.after(10, self.update)

    def process_frame(self, frame):
        # Initialize j
        j = 0

        # Resize frame for speedup
        frame = cv2.resize(frame, (self.frame_width, self.frame_height))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # YOLOv3 detection
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.net.getUnconnectedOutLayersNames())

        helmet_detected = False  # Flag to check if a helmet is detected

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == self.HELMET_CLASS_ID:
                    x, y, w, h = self.get_bbox_coordinates(detection)

                    # Check if width and height are positive
                    if w > 0 and h > 0:
                        helmet_roi = self.extract_helmet_roi(frame, x, y, w, h)

                        # Check if the helmet_roi is not empty
                        if not helmet_roi.size == 0:
                            prediction = self.predict_helmet_nonhelmet(helmet_roi)

                            label = "Helmet" if prediction[0] > 0.5 else "No Helmet"
                            color = (0, 255, 0) if label == "Helmet" else (0, 0, 255)
                            cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 2)
                            cv2.putText(frame_rgb, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                            helmet_detected = True

                            # Store the image if a bike is detected without a helmet and it's not already stored
                            if label == "No Helmet" and self.bike_detected.get(label) is None:
                                # Save the image using your provided code
                                name = self.video_source.split("\\")[-1].split(".")[0]  # Extract video name
                                j += 1
                                cv2.imwrite(f"{self.output_folder}/{name}_{str(j)}.jpg", frame)

        # ... (your existing code)

        return frame_rgb

    def get_bbox_coordinates(self, detection):
        center_x, center_y, w, h = int(detection[0] * self.frame_width), int(detection[1] * self.frame_height), \
                                   int(detection[2] * self.frame_width), int(detection[3] * self.frame_height)
        x, y = int(center_x - w / 2), int(center_y - h / 2)
        return x, y, w, h

    def extract_helmet_roi(self, frame, x, y, w, h):
        return frame[y:y + h, x:x + w]

    def predict_helmet_nonhelmet(self, helmet_roi):
        # Preprocess helmet_roi for helmet-nonhelmet model
        helmet_roi_resized = cv2.resize(helmet_roi, (224, 224))
        helmet_roi_normalized = helmet_roi_resized / 255.0
        helmet_roi_final = np.expand_dims(helmet_roi_normalized, axis=0)

        # Predict helmet or nonhelmet
        prediction = self.helmet_model.predict(helmet_roi_final)[0]
        return prediction

if __name__ == "__main__":
    app = HelmetDetectionApp(video_source="video.mp4")
'''

'import cv2\nimport numpy as np\nfrom keras.models import load_model\nimport tkinter as tk\nfrom PIL import Image, ImageTk\n\nclass HelmetDetectionApp:\n    """Application for detecting helmets in a video stream."""\n    HELMET_CLASS_ID = 0  # Update with the correct class ID for helmets\n\n    def __init__(self, video_source):\n        self.video_source = video_source\n        self.cap = cv2.VideoCapture(self.video_source)\n\n        self.frame_width = 500\n        self.frame_height = 500\n\n        # Load YOLOv3\n        self.net = cv2.dnn.readNet("yolov3-custom.cfg", "yolov3-custom_7000.weights")\n\n        # Load helmet-nonhelmet model\n        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")\n\n        self.root = tk.Tk()\n        self.root.title("Helmet Detection")\n\n        self.label = tk.Label(self.root)\n        self.label.pack()\n\n        # Folder to save snapshots\n        self.output_folder = "C:\\Users\\Asus\\Videos\\Captures\\New folder"\n\n        self.bike_d

In [9]:
import cv2
import numpy as np
from keras.models import load_model
import tkinter as tk
from PIL import Image, ImageTk

class HelmetDetectionApp:
    def __init__(self, video_source, output_video_path):
        self.video_source = video_source
        self.cap = cv2.VideoCapture(self.video_source)

        self.frame_width = 500
        self.frame_height = 500

        # Load helmet-nonhelmet model
        self.helmet_model = load_model("helmet-nonhelmet_cnn.h5")

        self.root = tk.Tk()
        self.root.title("Helmet Detection")

        self.label = tk.Label(self.root)
        self.label.pack()

        # Folder to save snapshots
        self.output_folder = r"C:\Users\Asus\Videos\Captures\New folder"

        # Video writer for saving processed video
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        self.output_video = cv2.VideoWriter(output_video_path, fourcc, 20.0, (self.frame_width, self.frame_height))

        self.bike_detected = {}  # Dictionary to track detected bikes

        self.update()

        self.root.mainloop()

    def update(self):
        ret, frame = self.cap.read()

        if ret:
            frame_rgb = self.process_frame(frame)

            # Convert the frame to a format compatible with Tkinter
            image = Image.fromarray(frame_rgb)
            image = ImageTk.PhotoImage(image=image)

            # Update the label with the new image
            self.label.config(image=image)
            self.label.image = image

            # Write the frame to the output video
            self.output_video.write(frame_rgb)

        else:
            # Release the video writer when the video ends
            self.output_video.release()

        # Schedule the next update after 10 milliseconds
        self.root.after(10, self.update)

    def process_frame(self, frame):
    # Resize frame for speedup
        frame = cv2.resize(frame, (self.frame_width, self.frame_height))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Preprocess frame for helmet-nonhelmet model
        frame_resized = cv2.resize(frame_rgb, (224, 224))
        frame_normalized = frame_resized / 255.0
        frame_final = np.expand_dims(frame_normalized, axis=0)

    # Predict helmet or nonhelmet
        prediction = self.helmet_model.predict(frame_final)[0]

    # Draw a bounding box based on the model prediction
        label = "Helmet" if prediction > 0.5 else "No Helmet"
        color = (0, 255, 0) if label == "Helmet" else (0, 0, 255)
        cv2.rectangle(frame_rgb, (0, 0), (self.frame_width, self.frame_height), color, 2)
        cv2.putText(frame_rgb, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Store the image if a bike is detected without a helmet
        if label == "No Helmet":
        # Save the image with a unique filename
            output_path = f"{self.output_folder}/no_helmet_{len(self.bike_detected)}.jpg"
            print(f"Saving image to: {output_path}")
            cv2.imwrite(output_path, frame)
            self.bike_detected[label] = True
    
        return frame_rgb

    


        return frame_rgb

if __name__ == "__main__":
    app = HelmetDetectionApp(video_source="video.mp4", output_video_path="processed_video.avi")


1/1 [==============================] - 0s 413ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_0.jpg
1/1 [==============================] - 0s 105ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 117ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 105ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 108ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 114ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 108ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 111ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/

1/1 [==============================] - 0s 47ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 49ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 47ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 58ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 47ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 51ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 57ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helmet_1.jpg
1/1 [==============================] - 0s 47ms/step
Saving image to: C:\Users\Asus\Videos\Captures\New folder/no_helme